In [2]:

import json
import pandas as pd

# Load the GeoJSON data
file_path = 'Hospitals SD.csv'  # Update the path if needed
file = pd.read_csv(file_path)
file.head()
file = file[file['CITY'] == 'SAN DIEGO']

file.to_csv("Hospitals_SD.csv")


In [17]:
# Load the GeoJSON data
file_path = 'Public_Schools SD.csv'  # Update the path if needed
file = pd.read_csv(file_path)
file.head()
file = file[file['CITY'] == 'SAN DIEGO']

file.to_csv("Schools_SD.csv")


In [25]:
zip_codes = set()
with open("San_Diego.geojson", 'r') as file:
    data = json.load(file)
    
    # Iterate through each feature in the GeoJSON data
    for feature in data['features']:
        # Extract the ZIP code from the properties
        zip_code = feature['properties'].get('ZIP')
        # Add the ZIP code to the set (sets automatically handle duplicates)
        if zip_code is not None:
            zip_codes.add(zip_code)
zip_codes.add(92037)

zip_codes.add(92014)
zip_codes.add(92065)
zip_codes.add(92064)
zip_codes.add(92173)

In [35]:
file = pd.read_csv("SDGE-ELEC-2023-Q4.csv")

file.head()
file = file[ file['ZipCode'].isin(zip_codes)]
df = file.groupby("ZipCode", group_keys=True)[['TotalkWh']].sum()
df = df[(df['TotalkWh'] != 0)]
df['TotalkWh'] = df['TotalkWh'] / 30


df.to_csv("Demand.csv")


In [106]:
df = pd.read_csv('Demand.csv')
hos = pd.read_csv('Hospitals_SD.csv')
result = pd.merge(df, hos, left_on='ZipCode', right_on='ZIP')
result = result.groupby("ZipCode", group_keys=True)[['OBJECTID']].count()
df = pd.merge(df, result, on='ZipCode', how='outer')

df.rename(columns={'OBJECTID': '#Hospitals'}, inplace=True)


In [107]:

school = pd.read_csv('Schools_SD.csv')
result = pd.merge(df, school, left_on='ZipCode', right_on='ZIP')
result
result = result.groupby("ZipCode", group_keys=True)[['OBJECTID']].count()
df = pd.merge(df, result, on='ZipCode', how='outer')
df.rename(columns={'OBJECTID': '#Schools'}, inplace=True)


In [108]:

stops = pd.read_csv("transit_stops_in_SD_county.csv")
temp = pd.merge(df, stops, left_on='ZipCode', right_on='Zipcode')
temp = temp.groupby("ZipCode", group_keys=True)[['stop_id']].count()
df = pd.merge(df, temp, on='ZipCode', how='outer')
df.rename(columns={'stop_id': '#Stops'}, inplace=True)


,ZipCode,TotalkWh,#Hospitals,#Schools,#Stops
0,92014,7.157900e+04,NaN,NaN,20.0
1,92037,1.726425e+05,NaN,NaN,14.0
2,92064,5.958945e+05,NaN,NaN,NaN
3,92065,1.864781e+05,NaN,NaN,19.0
4,92101,7.168670e+05,NaN,9.0,2.0
5,92102,1.949818e+05,NaN,19.0,NaN
6,92103,1.359799e+05,5.0,6.0,NaN
7,92104,1.353671e+05,1.0,4.0,NaN
8,92105,1.004162e+05,1.0,17.0,NaN
9,92106,5.070583e+04,NaN,12.0,NaN


In [109]:
col = pd.read_csv('Colleges_and_Universities.csv')
result = pd.merge(df, col, left_on='ZipCode', right_on='ZIP')
result
result = result.groupby("ZipCode", group_keys=True)[['OBJECTID']].count()
df = pd.merge(df, result, on='ZipCode', how='outer')
df.rename(columns={'OBJECTID': '#Colleges'}, inplace=True)

In [125]:

df.to_csv("Infrastructures.csv")

infra_data = pd.read_csv("Infrastructures.csv")
infra_data.fillna(0, inplace=True)

In [129]:
infra_dict = {}
for index, row in infra_data.iterrows():
    infra_dict[row['ZipCode']] = {
        'TotalkWh': row['TotalkWh'] / 24,  # Assuming you need to convert total kWh from daily to hourly
        'Hospitals': row['#Hospitals'],
        'Schools': row['#Schools'],  # Assuming this is the column name
        'Stops': row['#Stops'],    # Assuming this is the column name
        'Colleges': row['#Colleges']  # Assuming this is the column name
    }


{92014.0: {'TotalkWh': 2982.4583333333335,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 20.0,
  'Colleges': 0.0},
 92037.0: {'TotalkWh': 7193.438888888889,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 14.0,
  'Colleges': 3.0},
 92064.0: {'TotalkWh': 24828.93888888889,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 0.0,
  'Colleges': 2.0},
 92065.0: {'TotalkWh': 7769.920833333334,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 19.0,
  'Colleges': 0.0},
 92101.0: {'TotalkWh': 29869.458333333332,
  'Hospitals': 0.0,
  'Schools': 9.0,
  'Stops': 2.0,
  'Colleges': 12.0},
 92102.0: {'TotalkWh': 8124.240277777776,
  'Hospitals': 0.0,
  'Schools': 19.0,
  'Stops': 0.0,
  'Colleges': 1.0},
 92103.0: {'TotalkWh': 5665.829166666666,
  'Hospitals': 5.0,
  'Schools': 6.0,
  'Stops': 0.0,
  'Colleges': 0.0},
 92104.0: {'TotalkWh': 5640.297222222222,
  'Hospitals': 1.0,
  'Schools': 4.0,
  'Stops': 0.0,
  'Colleges': 0.0},
 92105.0: {'TotalkWh': 4184.006944444445,
  'Hospitals': 1.0,
  '

In [130]:
infrastructure = infra_dict
max_values = {
    'TotalkWh': max(info['TotalkWh'] for info in infrastructure.values()),
    'Hospitals': max(info['Hospitals'] for info in infrastructure.values()),
    'Schools': max(info['Schools'] for info in infrastructure.values()),
    'Stops': max(info['Stops'] for info in infrastructure.values()),
    'Colleges': max(info['Colleges'] for info in infrastructure.values())
}

# Step 2: Normalize each attribute by its respective maximum value
for zipcode, attributes in infrastructure.items():
    attributes['TotalkWh'] /= max_values['TotalkWh']
    attributes['Hospitals'] /= max_values['Hospitals'] if max_values['Hospitals'] != 0 else 1  # Avoid division by zero
    attributes['Schools'] /= max_values['Schools'] if max_values['Schools'] != 0 else 1  # Avoid division by zero
    attributes['Stops'] /= max_values['Stops'] if max_values['Stops'] != 0 else 1  # Avoid division by zero
    attributes['Colleges'] /= max_values['Colleges'] if max_values['Colleges'] != 0 else 1  # Avoid division by zero

# Output the normalized data

In [131]:
infrastructure

{92014.0: {'TotalkWh': 0.06935434159108524,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 1.0,
  'Colleges': 0.0},
 92037.0: {'TotalkWh': 0.1672768441854499,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 0.7,
  'Colleges': 0.25},
 92064.0: {'TotalkWh': 0.5773742720219663,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 0.0,
  'Colleges': 0.16666666666666666},
 92065.0: {'TotalkWh': 0.18068240470887484,
  'Hospitals': 0.0,
  'Schools': 0.0,
  'Stops': 0.95,
  'Colleges': 0.0},
 92101.0: {'TotalkWh': 0.6945869430053019,
  'Hospitals': 0.0,
  'Schools': 0.34615384615384615,
  'Stops': 0.1,
  'Colleges': 1.0},
 92102.0: {'TotalkWh': 0.18892177942459765,
  'Hospitals': 0.0,
  'Schools': 0.7307692307692307,
  'Stops': 0.0,
  'Colleges': 0.08333333333333333},
 92103.0: {'TotalkWh': 0.13175367683428954,
  'Hospitals': 0.7142857142857143,
  'Schools': 0.23076923076923078,
  'Stops': 0.0,
  'Colleges': 0.0},
 92104.0: {'TotalkWh': 0.13115995481085208,
  'Hospitals': 0.14285714285714285,
 

In [133]:
weights = {}
for key, value in infrastructure.items():
    weights[key] = 0.3*value["Hospitals"] + 0.3*value["Schools"] + 0.2*value["Stops"] + 0.2*value["Colleges"]
weights

{92014.0: 0.2,
 92037.0: 0.19,
 92064.0: 0.03333333333333333,
 92065.0: 0.19,
 92101.0: 0.32384615384615384,
 92102.0: 0.23589743589743586,
 92103.0: 0.2835164835164835,
 92104.0: 0.089010989010989,
 92105.0: 0.239010989010989,
 92106.0: 0.1717948717948718,
 92107.0: 0.046153846153846156,
 92108.0: 0.11153846153846154,
 92109.0: 0.06923076923076923,
 92110.0: 0.22029304029304025,
 92111.0: 0.3761904761904762,
 92113.0: 0.2012820512820513,
 92114.0: 0.16153846153846152,
 92115.0: 0.2115384615384615,
 92116.0: 0.057692307692307696,
 92117.0: 0.20769230769230768,
 92119.0: 0.046153846153846156,
 92120.0: 0.20934065934065932,
 92121.0: 0.12974358974358977,
 92122.0: 0.15076923076923077,
 92123.0: 0.48846153846153845,
 92124.0: 0.09230769230769231,
 92126.0: 0.14358974358974358,
 92127.0: 0.10384615384615384,
 92128.0: 0.11208791208791208,
 92129.0: 0.13846153846153847,
 92130.0: 0.16153846153846152,
 92131.0: 0.1205128205128205,
 92139.0: 0.08076923076923076,
 92154.0: 0.23076923076923078,

In [127]:
totalkwh_dict = dict(zip(infra_data['ZipCode'], infra_data['TotalkWh']/24))
totalkwh_dict = dict(zip(infra_data['ZipCode'], infra_data['#Hospitals']))
totalkwh_dict

{92014: 0.0,
 92037: 0.0,
 92064: 0.0,
 92065: 0.0,
 92101: 0.0,
 92102: 0.0,
 92103: 5.0,
 92104: 1.0,
 92105: 1.0,
 92106: 0.0,
 92107: 0.0,
 92108: 0.0,
 92109: 0.0,
 92110: 1.0,
 92111: 1.0,
 92113: 0.0,
 92114: 0.0,
 92115: 0.0,
 92116: 0.0,
 92117: 0.0,
 92119: 0.0,
 92120: 3.0,
 92121: 0.0,
 92122: 0.0,
 92123: 7.0,
 92124: 0.0,
 92126: 0.0,
 92127: 0.0,
 92128: 1.0,
 92129: 0.0,
 92130: 0.0,
 92131: 0.0,
 92139: 0.0,
 92154: 0.0,
 92173: 0.0}

In [18]:
file_path = 'National_Transit_Map_Stops SD.csv'  # Update the path if needed
file = pd.read_csv(file_path)
file.head()
file = file[file['CITY'] == 'SAN DIEGO']

file.to_csv("Transit_stops_SD.csv")

KeyError: 'CITY'

In [ ]:
with open(file_path, 'r') as file:
    geojson_data = json.load(file)

# Filter the GeoJSON data where 'COMMUNITY' is 'San Diego'
filtered_features = [feature for feature in geojson_data['features']
                     if feature['properties'].get('COMMUNITY') == 'San Diego']

# Create a new GeoJSON structure for the filtered data
filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features
}

# Save the filtered data to a new GeoJSON file
output_path = 'San_Diego.geojson'  # Update the path if needed
with open(output_path, 'w') as outfile:
    json.dump(filtered_geojson, outfile)
